In [77]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/corona_tested_individuals_ver_006.english.csv")
df.head()

C:\Users\marti\AppData\Local\Temp\ipykernel_36732\940471615.py:4: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/corona_tested_individuals_ver_006.english.csv")


,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2020-04-30,0,0,0,0,0,negative,None,female,Other
1,2020-04-30,1,0,0,0,0,negative,None,female,Other
2,2020-04-30,0,1,0,0,0,negative,None,male,Other
3,2020-04-30,1,0,0,0,0,negative,None,female,Other
4,2020-04-30,1,0,0,0,0,negative,None,male,Other


In [78]:
for col in ['cough','fever','sore_throat','shortness_of_breath','head_ache']:
 df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(np.int64)


In [79]:
df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2020-04-30,0,0,0,0,0,negative,None,female,Other
1,2020-04-30,1,0,0,0,0,negative,None,female,Other
2,2020-04-30,0,1,0,0,0,negative,None,male,Other
3,2020-04-30,1,0,0,0,0,negative,None,female,Other
4,2020-04-30,1,0,0,0,0,negative,None,male,Other


In [80]:
one_hot = pd.get_dummies(df.test_indication)
df.drop('test_indication', axis=1, inplace=True)
df = df.join(one_hot)
df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,Abroad,Contact with confirmed,Other
0,2020-04-30,0,0,0,0,0,negative,None,female,0,0,1
1,2020-04-30,1,0,0,0,0,negative,None,female,0,0,1
2,2020-04-30,0,1,0,0,0,negative,None,male,0,0,1
3,2020-04-30,1,0,0,0,0,negative,None,female,0,0,1
4,2020-04-30,1,0,0,0,0,negative,None,male,0,0,1


In [81]:
df.gender.value_counts()

female    130158
male      129127
None       19563
Name: gender, dtype: int64

In [82]:
df.corona_result = df.corona_result.map({"negative" : 0, "positive" : 1 , "other" : -1})

In [83]:
df.age_60_and_above = df.age_60_and_above.map({"No" : 0, "Yes" : 1 , "None" : -1})

In [84]:
df.gender = df.gender.map({"female" : 0, "male" : 1 , "None" : -1})

In [85]:
df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,Abroad,Contact with confirmed,Other
0,2020-04-30,0,0,0,0,0,0,-1,0,0,0,1
1,2020-04-30,1,0,0,0,0,0,-1,0,0,0,1
2,2020-04-30,0,1,0,0,0,0,-1,1,0,0,1
3,2020-04-30,1,0,0,0,0,0,-1,0,0,0,1
4,2020-04-30,1,0,0,0,0,0,-1,1,0,0,1


In [86]:
random_state=137
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [87]:
df = df.drop(df[df['corona_result']==-1].index)

In [88]:
X = df.drop(['test_date', 'corona_result'], axis=1)
y = df.corona_result

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=137)

In [90]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [91]:
y_pred = dtc.predict(X_test)

In [92]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.7863977189051115